In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

def post_to_facebook(fb_email: str,
                     fb_password: str,
                     image_path: str,
                     description: str,
                     headless: bool = False,
                     disable_notifications: bool = True):
    """
    Automatisation Selenium pour publier une image + description sur Facebook.
    
    :param fb_email:       Adresse e-mail ou numéro de téléphone Facebook.
    :param fb_password:    Mot de passe Facebook.
    :param image_path:     Chemin absolu vers le fichier image à poster.
    :param description:    Texte à publier.
    :param headless:       Si True, lance Chrome en mode headless.
    :param disable_notifications: Si True, bloque les popups notifications.
    """

    # === Configuration des options Chrome ===
    options = Options()
    if headless:
        options.add_argument("--headless")
    if disable_notifications:
        options.add_argument("--disable-notifications")

    # === Lancement du navigateur ===
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    wait = WebDriverWait(driver, 15)

    try:
        # === Accès et connexion ===
        driver.get("https://www.facebook.com")
        wait.until(EC.presence_of_element_located((By.ID, "email")))\
            .send_keys(fb_email)
        driver.find_element(By.ID, "pass").send_keys(fb_password)
        driver.find_element(By.NAME, "login").click()

        # === Attente de la page d’accueil ===
        wait.until(EC.url_contains("facebook.com"))
        time.sleep(3)

        # === Ouvrir le formulaire de publication ===
        post_button = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//span[contains(text(),'Quoi de neuf')]/ancestor::div[@role='button']")))
        post_button.click()

        # === Renseigner la description ===
        text_area = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//div[@role='textbox']")))
        text_area.send_keys(description)

        # === Cliquer sur "Photo/Vidéo" ===
        photo_button = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//div[@aria-label='Photo/Vidéo']")))
        photo_button.click()

        # === Uploader l’image ===
        file_input = wait.until(EC.presence_of_element_located(
            (By.XPATH, "//input[@type='file']")))
        file_input.send_keys(os.path.abspath(image_path))
        time.sleep(3)

        # === Publier ===
        publish_btn = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//div[@aria-label='Publier']")))
        publish_btn.click()

        # === Pause finale ===
        time.sleep(5)
        print("✅ Publication Facebook terminée avec succès !")

    except Exception as e:
        print(f"🚫 Échec de la publication Facebook : {e}")

    finally:
        driver.quit()



In [ ]:
#functionnal code
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

<function load_dotenv at 0x000002AAE213BBA0>


In [2]:
import google.generativeai as genai
import gradio as gr
from PIL import Image

# 🔐 Clé API Gemini
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

def est_description_finale(texte):
    # Exemple simple : contient des hashtags → description marketing finalisée
    return "#" in texte or texte.count(".") >= 3

chat_history = []
posts_history = []

image_stockée = None
description_stockée = None

# 🔹 Fonction 1 : Texte seul (style bulle WhatsApp)
def handle_image_and_text(user_message, image):
    global chat_history, posts_history, image_stockée, description_stockée

    if not user_message and not image:
        return "<div class='bot-bubble error'>Erreur : veuillez saisir un message ou ajouter une image.</div>"

    if user_message:
        chat_history.append(f"*الحرفي:* {user_message.strip()}")

    prompt = f"""
📌 Your role:
You are a smart assistant named Arti. You help Moroccan artisans promote and describe their traditional handmade products, especially through natural, friendly conversation.

📌 How to respond:
The user may write in:
- Arabic (e.g., "واش عندك صور؟")
- Or Darija using Latin characters (e.g., "salam kif dayr")

🔴 Very important rules:
↪ If the user types in *Latin Darija, respond in the **same format* (Latin Darija only).  
↪ If the user types in *Arabic script, respond in **Arabic only*.  
Never switch scripts.

📌 Expected behavior:
1. If the user uploads an image of a product:
2. If the user provides you with details about his product use them for the marketing caption.
3. Ask the user to choose a preferred language of the marketing description
    - 🇲🇦 Arabic (Darija or Classical Arabic)
    - 🇫🇷 French
    - 🇬🇧 English 
    or another language . 
4. Once the user selects the language, Generate a *marketing caption instagram and hashtags* in the chosen language  *do not give an opinion or emotional reaction*.
❌ Do not say things like "Superbe choix !" or "Le post est prêt" just resend the description selected.
5. Confirm that the post is ready ✅

📌 Conversation so far:

{chr(10).join(chat_history)}

*Arti:*"""

    try:
        model = genai.GenerativeModel("models/gemini-1.5-flash")
        if image:
            img = Image.open(image)
            response = model.generate_content([prompt, img])
        else:
            response = model.generate_content(prompt)

        bot_reply = response.text.strip()
        chat_history.append(f"*أنا:* {bot_reply}")

        # Sauvegarde unique : seulement si description finale et pas encore stockée
        if est_description_finale(bot_reply) and description_stockée is None:
            image_stockée = image
            description_stockée = bot_reply
            print("✅ Image stockée :", image_stockée)
            print("✅ Description générée :\n", description_stockée)

            # Sauvegarde dans l’historique
            posts_history.append({
                "image": image,
                "description": bot_reply
            })
            # 3) Publication Facebook
            try:
                post_to_facebook(
                    fb_email = os.getenv("FB_EMAIL"),
                    fb_password = os.getenv("FB_PASSWORD"),
                    image_path=image_stockée,
                    description=description_stockée,
                    headless=False
                )
                confirmation = "<div class='bot-bubble'>✅ Publication Facebook réussie !</div>"
            except Exception as e:
                confirmation = (
                    f"<div class='bot-bubble error'>"
                    f"🚫 Échec de la publication Facebook : {e}"
                    f"</div>"
                )
        # Affichage style WhatsApp
        formatted_html = ""
        for msg in chat_history:
            if msg.startswith("*الحرفي:*"):
                contenu = msg.replace("*الحرفي:*", "").strip()
                formatted_html += f"<div class='user-bubble'>{contenu}</div>"
            elif msg.startswith("*أنا:*"):
                contenu = msg.replace("*أنا:*", "").strip()
                formatted_html += f"<div class='bot-bubble'>{contenu}</div>"

        return formatted_html

    except Exception as e:
        return f"<div class='bot-bubble error'>[Erreur Gemini] : {str(e)}</div>"
# 🔸 Fonction 2 : Image + description
def analyze_image_and_generate(image, user_message):
    if not image:
        return """
        <style>
        .bot-bubble.error {
            background-color: #ffe0e0;
            color: #a00;
            border: 2px solid #f99;
            border-radius: 14px;
            padding: 15px;
            font-family: 'Cairo', sans-serif;
            direction: rtl;
        }
        </style>
        <div class='bot-bubble error'>🚫 المرجو تحميل صورة للمنتوج.</div>
        """

    image_pil = Image.open(image)
    cleaned_message = user_message.strip() if user_message else ""

    image_prompt = f"""
📌 الدور ديالك:
انت مساعد ذكي سميتكArtiBot. كتعاون الحرفيين المغاربة يوصفو منتوجاتهم التقليدية ويديرو التسويق ليها.

✅ المطلوب:
- وصف واضح للمنتوج لي فالصورة
- جملة قصيرة بحال caption ديال انستاغرام
- اقتراح hashtags
- جميع المعلومات المهمة لي تقدر تعاون الحرفي يبيع المنتوج

📸 راه الصورة قدامك{"، وها شنو قال المستخدم:" if cleaned_message else "، خدم غير على الصورة"}

{cleaned_message if cleaned_message else ""}

**أنا:**"""

    try:
        model = genai.GenerativeModel("models/gemini-1.5-flash")
        response = model.generate_content([image_pil, image_prompt])
        bot_reply = response.text.strip()

        return f"""
        <style>
        .bot-bubble {{
            background-color: #fff9f3;
            color: #3b2e1a;
            padding: 20px 25px;
            border-radius: 18px;
            margin: 15px auto 20px 40px;
            max-width: 95%;
            text-align: right;
            font-size: 16px;
            line-height: 1.8;
            border: 2px solid #e0b076;
            box-shadow: 0 3px 8px rgba(0,0,0,0.05);
            direction: rtl;
            white-space: pre-line;
            font-family: 'Cairo', sans-serif;
        }}

        .bot-bubble::before {{
            content: "🤖ArtiBot";
            display: block;
            font-weight: bold;
            color: #a44e00;
            margin-bottom: 10px;
            font-size: 18px;
        }}

        .bot-bubble strong {{
            color: #c16e0d;
        }}
        </style>

        <div class="bot-bubble">{bot_reply}</div>
        """

    except Exception as e:
        return f"""
        <style>
        .bot-bubble.error {{
            background-color: #ffe0e0;
            color: #a00;
            border: 2px solid #f99;
            border-radius: 14px;
            padding: 15px;
            font-family: 'Cairo', sans-serif;
            direction: rtl;
        }}
        </style>
        <div class='bot-bubble error'>[Erreur Gemini] : {str(e)}</div>
        """

# 🎨 Interface Gradio avec CSS stylé
with gr.Blocks(css="""
body { background-color: #fcfcfc; font-family: 'Segoe UI', sans-serif; }
.gr-button { background: linear-gradient(to right, #ff8c00, #ff6600); color: white; font-weight: bold; border-radius: 8px; }
.gr-textbox textarea { border: 2px solid #ffcc99; border-radius: 10px; padding: 10px; background-color: #fffbe6; }
.gr-image { border-radius: 10px; border: 2px dashed #ffa366; }
.gr-markdown h1, h2 { color: #ff6600; }
.user-bubble {
    background-color: #d4f8d4;
    color: #000;
    padding: 10px 15px;
    border-radius: 15px;
    margin: 8px 40px 8px auto;
    max-width: 80%;
    text-align: left;
}
.bot-bubble {
    background-color: #f1f0f0;
    color: #000;
    padding: 10px 15px;
    border-radius: 15px;
    margin: 8px auto 8px 40px;
    max-width: 80%;
    text-align: left;
}
.error {
    background-color: #ffe0e0;
    color: #a00;
    border: 1px solid #f99;
}
""") as demo:
    

    gr.Markdown("🧵 **Arti – Promote Your Craft with AI 🇲🇦**")

    with gr.Tab("📝 Post Generation"):
        with gr.Row():
            text_input = gr.Textbox(label="✍️ Dis quelque chose sur ton produit (Darija ou arabe)", scale=5)
            image_input = gr.Image(label="📎 Image du produit", type="filepath", scale=2)

        run_btn = gr.Button("✨ Générer la réponse")
        output_text = gr.HTML()

        run_btn.click(
            fn=handle_image_and_text,
            inputs=[text_input, image_input],
            outputs=output_text
        )

    with gr.Tab("📸 Image + Description"):
        with gr.Row():
            image_input = gr.Image(type="filepath", label="📸 Uploade ton image")
            text_hint = gr.Textbox(label="✍️ (Optionnel) Donne une petite description")
        run_img = gr.Button("🎨 Générer à partir de l’image")
        output_img = gr.HTML()
        run_img.click(fn=analyze_image_and_generate, inputs=[image_input, text_hint], outputs=output_img)

demo.launch(share=True)



* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Missing file: C:\Users\DELL\.cache\huggingface\gradio\frpc\frpc_windows_amd64_v0.3. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_windows_amd64.exe
2. Rename the downloaded file to: frpc_windows_amd64_v0.3
3. Move the file to this location: C:\Users\DELL\.cache\huggingface\gradio\frpc
